In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import optimizers

Using TensorFlow backend.


# Preprocessing Data

In [2]:
df = pd.read_csv('data/skeleton_filtered.csv')

df

,nose_x,nose_y,neck_x,neck_y,Rshoulder_x,Rshoulder_y,Relbow_x,Relbow_y,Rwrist_x,RWrist_y,...,LAnkle_y,REye_x,REye_y,LEye_x,LEye_y,REar_x,REar_y,LEar_x,LEar_y,class
0,0.643519,0.157609,0.634259,0.250000,0.587963,0.244565,0.555556,0.342391,0.550926,0.434783,...,0.755435,0.629630,0.146739,0.652778,0.146739,0.615741,0.157609,0.666667,0.152174,kick
1,0.643519,0.157609,0.634259,0.250000,0.587963,0.244565,0.560185,0.342391,0.555556,0.429348,...,0.766304,0.629630,0.146739,0.652778,0.146739,0.615741,0.157609,0.666667,0.152174,kick
2,0.643519,0.157609,0.638889,0.250000,0.587963,0.244565,0.560185,0.342391,0.555556,0.429348,...,0.760870,0.634259,0.146739,0.652778,0.146739,0.615741,0.157609,0.671296,0.152174,kick
3,0.643519,0.157609,0.638889,0.250000,0.587963,0.244565,0.560185,0.342391,0.555556,0.429348,...,0.755435,0.629630,0.146739,0.652778,0.146739,0.615741,0.157609,0.666667,0.152174,kick
4,0.643519,0.157609,0.638889,0.250000,0.587963,0.244565,0.560185,0.342391,0.555556,0.429348,...,0.760870,0.629630,0.146739,0.652778,0.146739,0.615741,0.157609,0.666667,0.152174,kick
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3219,0.587963,0.152174,0.587963,0.233696,0.541667,0.217391,0.500000,0.163043,0.486111,0.092391,...,0.755435,0.578704,0.141304,0.597222,0.141304,0.564815,0.152174,0.620370,0.146739,wave
3220,0.583333,0.157609,0.587963,0.233696,0.541667,0.222826,0.490741,0.179348,0.458333,0.114130,...,0.755435,0.574074,0.141304,0.597222,0.141304,0.564815,0.152174,0.615741,0.146739,wave
3221,0.583333,0.157609,0.587963,0.228261,0.537037,0.222826,0.481481,0.179348,0.439815,0.108696,...,0.755435,0.574074,0.141304,0.000000,0.000000,0.560185,0.146739,0.615741,0.146739,wave
3222,0.578704,0.157609,0.583333,0.228261,0.532407,0.222826,0.476852,0.184783,0.421296,0.125000,...,0.755435,0.569444,0.146739,0.592593,0.141304,0.560185,0.152174,0.611111,0.146739,wave


In [3]:
# activity = df['class'].drop_duplicates().to_list()
activity = ['kick', 'stand', 'wave']

activity

['kick', 'stand', 'wave']

In [4]:
unique, counts = np.unique(df['class'], return_counts=True)
itemfreq = np.vstack((unique, counts)).T

itemfreq

array([['kick', 525],
       ['punch', 438],
       ['squat', 571],
       ['stand', 861],
       ['wave', 829]], dtype=object)

In [5]:
def process_create_WindowTimeSeries(activity, df, activity_start, activity_len, time_window, n_feature, step_stride):
    df_series = df[df['class'] == activity].iloc[:,:-1][activity_start:(activity_start+activity_len)]
    segments = []
    labels = []

    for i in range(0, df_series.shape[0] - time_window, step_stride):
        df_series_feature = df_series.iloc[i: i + time_window]
        label = activity

        segments.append(np.array(df_series_feature))
        labels.append(label)
        
    segments = np.asarray(segments)
    labels = np.asarray(labels)

    print(segments.shape, labels.shape)
#     reshaped_segments = segments.reshape(-1, time_window, n_feature, 1)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_window, n_feature,1)
    print(reshaped_segments.shape, labels.shape)

    return reshaped_segments, labels

In [6]:
activity_start = 0
activity_len = df.shape[0]
time_window = 100
n_feature = len(activity)
step_stride = 10

X = []
y = []

for i in activity:
    reshaped_segments, labels = process_create_WindowTimeSeries(i, df, activity_start, activity_len, time_window, n_feature, step_stride)
    X.append(reshaped_segments)
    y.append(labels)
    
X = np.concatenate(X)
y = np.concatenate(y)

X.shape, y.shape

(43, 100, 36) (43,)
(516, 100, 3, 1) (43,)
(77, 100, 36) (77,)
(924, 100, 3, 1) (77,)
(73, 100, 36) (73,)
(876, 100, 3, 1) (73,)


((2316, 100, 3, 1), (193,))

In [7]:
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)

dummy_y = np.zeros((encoded_y.shape[0], len(activity)))
dummy_y[np.arange(encoded_y.shape[0]), encoded_y] = 1

In [8]:
train_size = 0.8
test_size = 0.2
random_state = 0

X_train, X_validate, y_train, y_validate = train_test_split(X, dummy_y, train_size=train_size, test_size=test_size, random_state=random_state, stratify=y)

X_train.shape, X_validate.shape, y_train.shape, y_validate.shape

ValueError: Found input variables with inconsistent numbers of samples: [2316, 193]